In [1]:
import sys
sys.path.append('..')

from data_parsers import databento_file_parser
from simulator import SimpleSingleTickerSimulator

import pandas as pd
import math

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 80)

In [2]:
data_file_path = '../data/databento/AMZN/xnas-itch-20230508.mbp-10.dbn.zst'
amzn_df = databento_file_parser(data_file_path)
amzn_df

,ts_event,action,side,depth,price,size,flags,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_px_01,ask_px_01,bid_sz_01,ask_sz_01,bid_px_02,ask_px_02,bid_sz_02,ask_sz_02,bid_px_03,ask_px_03,bid_sz_03,ask_sz_03,bid_px_04,ask_px_04,bid_sz_04,ask_sz_04,bid_px_05,ask_px_05,bid_sz_05,ask_sz_05,bid_px_06,ask_px_06,bid_sz_06,ask_sz_06,bid_px_07,ask_px_07,bid_sz_07,ask_sz_07,bid_px_08,ask_px_08,bid_sz_08,ask_sz_08,bid_px_09,ask_px_09,bid_sz_09,ask_sz_09,symbol
ts_recv,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-05-08 14:30:00.006929087,2023-05-08 14:30:00.006765120,C,N,0,105.30,100,130,105.30,105.31,600,100,105.29,105.32,479,608,105.28,105.33,953,1600,105.27,105.34,475,794,105.26,105.35,425,384,105.25,105.36,1275,951,105.24,105.37,600,875,105.23,105.38,814,283,105.22,105.39,172,1269,105.21,105.40,842,283,AMZN
2023-05-08 14:30:00.006942603,2023-05-08 14:30:00.006778449,C,N,1,105.29,100,130,105.30,105.31,600,100,105.29,105.32,379,608,105.28,105.33,953,1600,105.27,105.34,475,794,105.26,105.35,425,384,105.25,105.36,1275,951,105.24,105.37,600,875,105.23,105.38,814,283,105.22,105.39,172,1269,105.21,105.40,842,283,AMZN
2023-05-08 14:30:00.049863902,2023-05-08 14:30:00.049699564,A,N,0,105.31,100,130,105.30,105.31,600,200,105.29,105.32,379,608,105.28,105.33,953,1600,105.27,105.34,475,794,105.26,105.35,425,384,105.25,105.36,1275,951,105.24,105.37,600,875,105.23,105.38,814,283,105.22,105.39,172,1269,105.21,105.40,842,283,AMZN
2023-05-08 14:30:00.049868386,2023-05-08 14:30:00.049703780,C,N,0,105.30,100,130,105.30,105.31,500,200,105.29,105.32,379,608,105.28,105.33,953,1600,105.27,105.34,475,794,105.26,105.35,425,384,105.25,105.36,1275,951,105.24,105.37,600,875,105.23,105.38,814,283,105.22,105.39,172,1269,105.21,105.40,842,283,AMZN
2023-05-08 14:30:00.049890420,2023-05-08 14:30:00.049725052,C,N,2,105.28,100,130,105.30,105.31,500,200,105.29,105.32,379,608,105.28,105.33,853,1600,105.27,105.34,475,794,105.26,105.35,425,384,105.25,105.36,1275,951,105.24,105.37,600,875,105.23,105.38,814,283,105.22,105.39,172,1269,105.21,105.40,842,283,AMZN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-08 20:59:57.047924972,2023-05-08 20:59:57.047760707,C,N,0,105.67,6,130,105.65,105.68,2000,100,105.60,105.70,121,633,105.57,105.72,30,100,105.55,105.75,100,50,105.53,105.80,28,2,105.52,105.84,10,16,105.51,105.90,100,58,105.50,105.92,139,100,105.45,105.96,100,10,105.44,105.97,2,400,AMZN
2023-05-08 20:59:57.047958552,2023-05-08 20:59:57.047794809,A,N,0,105.66,100,130,105.66,105.68,100,100,105.65,105.70,2000,633,105.60,105.72,121,100,105.57,105.75,30,50,105.55,105.80,100,2,105.53,105.84,28,16,105.52,105.90,10,58,105.51,105.92,100,100,105.50,105.96,139,10,105.45,105.97,100,400,AMZN
2023-05-08 20:59:59.255118037,2023-05-08 20:59:59.254954329,A,N,0,105.66,100,130,105.66,105.68,200,100,105.65,105.70,2000,633,105.60,105.72,121,100,105.57,105.75,30,50,105.55,105.80,100,2,105.53,105.84,28,16,105.52,105.90,10,58,105.51,105.92,100,100,105.50,105.96,139,10,105.45,105.97,100,400,AMZN


In [3]:
# Sample BOT to show the interface
class SampleBOT1(SimpleSingleTickerSimulator):
    
    # Place 1 order at level 2 with 10% of the volume of the level. 
    # Cancel if it is outside the 10 levels of the orderbook 
    def run_algo(self, bid_orderbook, ask_orderbook, inventory, bid_orders, ask_orders):
        
        if len(ask_orders) == 0:
            
            price, size = ask_orderbook[2]
            size = int(size/10)            
            if size > 0:
                self.place_order(price, size, side='ASK')
        else:
            order_price = list(ask_orders.keys())[0]
            if order_price > ask_orderbook[9][0] or order_price < ask_orderbook[0][0]:
                self.cancel_order(order_price, side='ASK')

        if len(bid_orders) == 0:
            
            price, size = bid_orderbook[2]
            size = int(size/10)
            if size > 0:
                self.place_order(price, size, side='BID')
        
        else:
            order_price = list(bid_orders.keys())[0]
            if order_price < bid_orderbook[9][0] or order_price > bid_orderbook[0][0]:
                self.cancel_order(order_price, side='BID')

In [4]:
bot = SampleBOT1(ticker='AMZN')
bot.run_sim(amzn_df)

self.current_ts=Timestamp('2023-05-08 14:35:34.912718464') 
self.BID_SIM_ORDER_BOOK={0: (105.1, 400), 1: (105.09, 860), 2: (105.08, 1000), 3: (105.07, 1400), 4: (105.06, 1125), 5: (105.05, 625), 6: (105.04, 1175), 7: (105.03, 632), 8: (105.02, 1611), 9: (105.01, 326)} 
self.ASK_SIM_ORDER_BOOK={0: (105.11, 114), 1: (105.12, 290), 2: (105.13, 325), 3: (105.14, 525), 4: (105.15, 425), 5: (105.16, 650), 6: (105.17, 182), 7: (105.18, 1057), 8: (105.19, 282), 9: (105.2, 1386)} 
self.BID_ALGO_ORDERS={105.03: 57} 
self.ASK_ALGO_ORDERS={105.11: 39} 
self.ALGO_POSITION=287
self.current_ts=Timestamp('2023-05-08 14:41:17.923654402') 
self.BID_SIM_ORDER_BOOK={0: (105.34, 670), 1: (105.33, 1237), 2: (105.32, 810), 3: (105.31, 1229), 4: (105.3, 1034), 5: (105.29, 975), 6: (105.28, 772), 7: (105.27, 552), 8: (105.26, 494), 9: (105.25, 229)} 
self.ASK_SIM_ORDER_BOOK={0: (105.35, 600), 1: (105.36, 685), 2: (105.37, 425), 3: (105.38, 177), 4: (105.39, 425), 5: (105.4, 1371), 6: (105.41, 625), 7: (105.42,

In [7]:
print(bot.ALGO_FILLS)

[(105.36, -1), (105.36, -73), (105.36, -16), (105.3, 2), (105.3, 28), (105.3, 12), (105.37, -3), (105.37, -10), (105.37, -3), (105.35, 4), (105.29, 2), (105.35, -24), (105.35, -1), (105.34, 32), (105.34, 2), (105.34, 27), (105.34, 21), (105.34, 4), (105.34, 49), (105.26, 37), (105.26, 14), (105.26, 8), (105.23, 1), (105.23, 19), (105.23, 1), (105.23, 21), (105.25, -17), (105.25, -13), (105.25, -21), (105.2, 1), (105.2, 3), (105.2, 1), (105.2, 6), (105.2, 1), (105.2, 4), (105.2, 10), (105.2, 5), (105.2, 12), (105.2, 14), (105.2, 21), (105.2, 1), (105.2, 1), (105.14, 23), (105.14, 14), (105.11, 1), (105.11, 1), (105.11, 5), (105.11, 6), (105.12, -48), (105.15, -7), (105.16, 38), (105.13, 32), (105.04, 36), (105.04, 1), (105.01, 23), (105.08, -20), (105.14, -21), (105.14, -4), (105.14, -12), (105.2, -1), (105.2, -68), (105.23, -1), (105.24, 12), (105.24, 10), (105.24, 9), (105.24, 8), (105.24, 28), (105.24, 4), (105.24, 4), (105.29, -1), (105.29, -5), (105.29, -5), (105.29, -8), (105.29, 

In [8]:
capital = 0
position = 0
for fill in bot.ALGO_FILLS:
    capital -= fill[0] * fill[1]
    position += fill[1]
    
print(capital, position)
pnl = capital + position * bot.ALGO_FILLS[-1][0]
print(pnl)

314693.2099999995 -2969
840.2199999995064
